# RAGAS 向量检索评估教程

## 目录
1. [RAGAS框架概述](#概述)
2. [安装和环境配置](#安装)
3. [RAGAS评估指标详解](#指标详解)
4. [创建RAGAS评估链](#评估链)
5. [单个结果评估](#单个评估)
6. [批量评估实战](#批量评估)
7. [向量检索专项评估](#检索评估)
8. [完整示例代码](#完整示例)
9. [结果分析和优化建议](#分析优化)


## 1. RAGAS框架概述 {#概述}

RAGAS（Retrieval Augmented Generation Assessment Suite）是一个专门为RAG系统设计的评估框架。

### 核心特点：
- ✅ **无参考评估**：不需要标准答案就能评估RAG系统
- ✅ **多维度指标**：覆盖检索、生成、端到端性能
- ✅ **LangChain集成**：可以与LangChain无缝集成
- ✅ **自动化评估**：使用LLM自动评估，减少人工标注成本

### RAGAS评估流程：
```
用户问题 → RAG系统 → 检索文档 + 生成答案
                          ↓
                    RAGAS评估器
                          ↓
              Faithfulness + Answer Relevancy
              + Context Precision + Context Recall
                          ↓
                    评估报告
```


In [ ]:
# 配置环境变量（如果使用OpenAI）
import os
# os.environ["OPENAI_API_KEY"] = "your-api-key-here"
import langchain_core
import langchain_openai
import ragas

from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall
)
from ragas import evaluate, Dataset
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
import pandas as pd
import numpy as np

## 3. RAGAS评估指标详解 {#指标详解}

RAGAS提供了四个核心评估指标：

### 3.1 Faithfulness（忠实度）
**定义**：评估生成的答案是否忠实于检索到的文档，即答案是否基于文档内容生成，没有引入幻觉。

**评估方法**：
1. 从答案中提取可验证的事实声明
2. 检查这些声明是否可以从检索文档中推断出来
3. 计算可验证声明的比例

**取值范围**：0-1，越接近1表示答案越忠实于文档

### 3.2 Answer Relevancy（答案相关性）
**定义**：评估生成的答案与用户问题的相关性。

**评估方法**：
1. 从答案中提取关键信息
2. 评估这些信息回答用户问题的程度
3. 考虑答案的完整性和相关性

**取值范围**：0-1，越接近1表示答案越相关

### 3.3 Context Precision（上下文精确度）
**定义**：评估检索到的文档中有多少是真正相关的。

**评估方法**：
1. 从答案中提取相关信息
2. 评估每个检索文档对这些信息的相关性
3. 计算相关文档在前K个结果中的比例

**取值范围**：0-1，越接近1表示检索越精确

### 3.4 Context Recall（上下文召回率）
**定义**：评估所有相关文档中有多少被成功检索到。

**评估方法**：
1. 使用标准答案（ground truth）作为参考
2. 评估检索到的文档是否包含标准答案中的信息
3. 计算覆盖的信息比例

**取值范围**：0-1，越接近1表示召回越完整

**注意**：Context Recall需要ground truth作为参考，其他三个指标是无需参考的。


## 4. 创建RAGAS评估指标 {#评估链}

RAGAS新版本（0.1.0+）使用`evaluate()`函数和`Dataset`来进行评估，不再使用`RagasEvaluatorChain`。我们可以直接使用评估指标和`evaluate()`函数。


In [ ]:
# 定义评估指标列表
# 新版本的RAGAS直接使用evaluate()函数，不再需要创建评估链

# 确保导入已执行（如果未导入，尝试导入）
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall
)
from ragas import evaluate, Dataset

metrics = [
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall
]

# 使用列表索引来映射指标对象到名称（因为指标对象不可哈希，不能用作字典键）
metric_names_list = [
    "faithfulness",
    "answer_relevancy",
    "context_precision",
    "context_recall"
]

# 创建一个辅助函数来获取指标名称
def get_metric_name(metric):
    """根据指标对象获取其名称"""
    try:
        idx = metrics.index(metric)
        return metric_names_list[idx]
    except ValueError:
        # 如果找不到，尝试使用字符串表示
        return str(metric).lower().replace(" ", "_")

print("已准备以下评估指标：")
for i, metric in enumerate(metrics):
    print(f"  - {metric_names_list[i]}")


## 5. 单个结果评估 {#单个评估}

首先，我们需要创建一个简单的RAG系统作为示例，然后对其进行评估。


In [ ]:
# 使用 HuggingFace Embeddings（更稳定，避免 modelscope 依赖问题）
from langchain_openai import ChatOpenAI
from langchain_community.embeddings import HuggingFaceEmbeddings

def create_simple_rag_system():
    """
    创建一个简单的RAG系统，使用 HuggingFace 的中文 embedding 模型
    在实际使用中，你需要替换为你的RAG系统
    """
    # 示例：使用OpenAI模型
    llm = ChatOpenAI(model="gpt-4o", api_key='', base_url='', temperature=0, max_tokens=1000)
    
    # 使用 HuggingFace 的 BGE-M3 中文模型（自动下载到本地）
    embeddings = HuggingFaceEmbeddings(
        model_name="BAAI/bge-small-zh-v1.5",  # 中文小模型，速度快
        model_kwargs={'device': 'cpu'},
        encode_kwargs={'normalize_embeddings': True}
    )

    # 创建示例文档（实际应该从向量库加载）
    example_docs = [
        "RAG（Retrieval-Augmented Generation）是一种结合检索和生成的技术。",
        "RAG的核心思想是在生成答案前，先从知识库中检索相关文档。",
        "RAG可以解决大模型的知识盲区问题，让模型能够访问非公开知识。"
    ]
    
    # 创建模板
    from langchain.prompts import PromptTemplate
    template = """
    根据以下文档回答问题：

    文档：
    {context}

    问题：{question}

    答案：
    """
    prompt = PromptTemplate.from_template(template)
    
    # 返回一个简单的RAG函数
    def rag_bot(question: str, documents: list = None):
        if documents is None:
            documents = example_docs
        
        context = "\n".join(documents)
        # 实际应用中，这里应该用 LLM 和 embedding 检索
        # answer = llm.invoke(prompt.format(context=context, question=question))
        answer = "RAG是一种结合检索和生成的技术，它通过在生成答案前从知识库检索相关文档来解决大模型的知识盲区问题。"

        return {
            "question": question,
            "answer": answer,
            "contexts": documents,
            "ground_truths": [answer]  # 示例ground truth
        }
    
    return rag_bot

# 创建RAG系统
rag_bot = create_simple_rag_system()
print("===== RAG系统已创建（使用 HuggingFace BGE 中文模型）=====")

In [ ]:
# 执行单个查询并获取结果
question = "RAG是什么？"
result = rag_bot(question)

print("=" * 50)
print("RAG系统输出：")
print("=" * 50)
print(f"问题：{result['question']}")
print(f"\n答案：{result['answer']}")
print(f"\n检索到的文档数量：{len(result['contexts'])}")
print(f"\n检索文档：")
for i, doc in enumerate(result['contexts'], 1):
    print(f"  {i}. {doc[:100]}...")


In [ ]:
# 使用RAGAS评估单个结果
print("=" * 50)
print("RAGAS评估结果：")
print("=" * 50)

# 新版本的RAGAS使用Dataset和evaluate()函数
# 准备评估数据：Dataset需要包含question, answer, contexts, ground_truths字段
eval_data = {
    "question": [result["question"]],
    "answer": [result["answer"]],
    "contexts": [result["contexts"]],
    "ground_truths": [result.get("ground_truths", [result["answer"]])]
}

# 创建Dataset对象
dataset = Dataset.from_dict(eval_data)

# 执行评估（包含所有指标）
# 注意：context_recall需要ground_truths，其他指标不需要
evaluation_result = evaluate(
    dataset=dataset,
    metrics=metrics
)

# 提取评估结果
print("\n评估分数：")
evaluation_results = {}
for metric in metrics:
    metric_name = get_metric_name(metric)  # 使用辅助函数获取指标名称
    try:
        score = evaluation_result[metric_name]
        evaluation_results[metric_name] = score
        print(f"{metric_name:20s}: {score:.4f}" if isinstance(score, (int, float)) else f"{metric_name:20s}: {score}")
    except Exception as e:
        print(f"{metric_name:20s}: 评估失败 - {str(e)}")
        evaluation_results[metric_name] = None


## 6. 批量评估实战 {#批量评估}

在实际应用中，我们需要对多个测试用例进行批量评估，以全面了解RAG系统的性能。


In [ ]:
# 准备测试数据集
test_dataset = [
    {
        "question": "RAG是什么？",
        "ground_truths": ["RAG是一种结合检索和生成的技术，用于解决大模型的知识盲区问题。"],
        "relevant_docs": [
            "RAG（Retrieval-Augmented Generation）是一种结合检索和生成的技术。",
            "RAG的核心思想是在生成答案前，先从知识库中检索相关文档。",
            "RAG可以解决大模型的知识盲区问题，让模型能够访问非公开知识。"
        ]
    },
    {
        "question": "RAG的核心思想是什么？",
        "ground_truths": ["RAG的核心思想是在生成答案前先从知识库检索相关文档。"],
        "relevant_docs": [
            "RAG的核心思想是在生成答案前，先从知识库中检索相关文档。",
            "检索到的文档作为上下文输入给生成模型，帮助生成更准确的答案。"
        ]
    },
    {
        "question": "RAG能解决什么问题？",
        "ground_truths": ["RAG可以解决大模型的知识盲区问题，让模型能够访问非公开知识。"],
        "relevant_docs": [
            "RAG可以解决大模型的知识盲区问题，让模型能够访问非公开知识。",
            "通过RAG，模型可以动态检索最新的信息，而不需要重新训练。"
        ]
    }
]

print(f"测试数据集大小：{len(test_dataset)}")
print("\n数据集示例：")
for i, item in enumerate(test_dataset[:2], 1):
    print(f"\n{i}. 问题：{item['question']}")
    print(f"   相关文档数：{len(item['relevant_docs'])}")


In [ ]:
# 批量评估函数
def batch_evaluate_ragas(rag_system, test_dataset, metrics_list):
    """
    批量评估RAG系统
    
    Args:
        rag_system: RAG系统函数
        test_dataset: 测试数据集列表
        metrics_list: 评估指标列表
    
    Returns:
        DataFrame: 包含所有评估结果的DataFrame
    """
    # 收集所有RAG系统输出
    all_questions = []
    all_answers = []
    all_contexts = []
    all_ground_truths = []
    
    print("收集RAG系统输出...")
    for idx, test_case in enumerate(test_dataset):
        print(f"  处理测试用例 {idx + 1}/{len(test_dataset)}: {test_case['question']}")
        
        # 获取RAG系统输出
        result = rag_system(
            question=test_case["question"],
            documents=test_case.get("relevant_docs", [])
        )
        
        all_questions.append(result["question"])
        all_answers.append(result["answer"])
        all_contexts.append(result["contexts"])
        all_ground_truths.append(test_case.get("ground_truths", [result["answer"]]))
    
    # 创建Dataset对象
    print("\n创建评估数据集...")
    eval_data = {
        "question": all_questions,
        "answer": all_answers,
        "contexts": all_contexts,
        "ground_truths": all_ground_truths
    }
    dataset = Dataset.from_dict(eval_data)
    
    # 执行批量评估
    print("执行评估...")
    evaluation_result = evaluate(
        dataset=dataset,
        metrics=metrics_list
    )
    
    # 转换为DataFrame格式
    results = []
    for idx in range(len(test_dataset)):
        case_result = {
            "question": all_questions[idx],
            "answer": all_answers[idx]
        }
        
        # 提取每个指标的分数
        for metric in metrics_list:
            metric_name = get_metric_name(metric)  # 使用辅助函数获取指标名称
            try:
                # 评估结果是一个字典，每个指标对应一个数组
                scores = evaluation_result[metric_name]
                case_result[metric_name] = scores[idx] if idx < len(scores) else None
            except Exception as e:
                print(f"  提取 {metric_name} 分数时出错: {str(e)}")
                case_result[metric_name] = None
        
        results.append(case_result)
    
    return pd.DataFrame(results)

# 执行批量评估
print("开始批量评估...")
evaluation_df = batch_evaluate_ragas(rag_bot, test_dataset, metrics)
print("\n评估完成！")


In [ ]:
# 查看评估结果
print("=" * 80)
print("批量评估结果：")
print("=" * 80)
print(evaluation_df.to_string(index=False))

# 计算平均分数
print("\n" + "=" * 80)
print("平均评估分数：")
print("=" * 80)
metric_columns = ['faithfulness', 'answer_relevancy', 'context_precision', 'context_recall']
for metric in metric_columns:
    if metric in evaluation_df.columns:
        avg_score = evaluation_df[metric].mean()
        print(f"{metric:20s}: {avg_score:.4f}")


In [ ]:
# 可视化评估结果
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']  # 支持中文
matplotlib.rcParams['axes.unicode_minus'] = False

# 绘制评估分数对比图
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 1. 各指标平均分数柱状图
metric_means = []
metric_names = []
for metric in metric_columns:
    if metric in evaluation_df.columns:
        metric_means.append(evaluation_df[metric].mean())
        metric_names.append(metric.replace('_', ' ').title())

axes[0].bar(metric_names, metric_means, color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'])
axes[0].set_title('RAGAS评估指标平均分数', fontsize=14, fontweight='bold')
axes[0].set_ylabel('分数', fontsize=12)
axes[0].set_ylim([0, 1])
axes[0].grid(axis='y', alpha=0.3)

# 添加数值标签
for i, v in enumerate(metric_means):
    axes[0].text(i, v + 0.02, f'{v:.3f}', ha='center', va='bottom', fontsize=10)

# 2. 每个测试用例的分数趋势
x = range(len(evaluation_df))
for metric in metric_columns:
    if metric in evaluation_df.columns:
        axes[1].plot(x, evaluation_df[metric], marker='o', label=metric.replace('_', ' ').title(), linewidth=2)

axes[1].set_title('各测试用例评估分数趋势', fontsize=14, fontweight='bold')
axes[1].set_xlabel('测试用例编号', fontsize=12)
axes[1].set_ylabel('分数', fontsize=12)
axes[1].set_xticks(x)
axes[1].set_xticklabels([f'Case {i+1}' for i in x])
axes[1].legend(loc='best')
axes[1].grid(alpha=0.3)
axes[1].set_ylim([0, 1])

plt.tight_layout()
plt.show()


## 7. 向量检索专项评估 {#检索评估}

对于向量检索系统，Context Precision和Context Recall是最重要的指标。本节详细介绍如何使用这两个指标评估检索质量。


In [ ]:
# 专门用于评估检索质量的函数
def evaluate_retrieval_quality(
    questions: list,
    retrieval_function,
    ground_truth_docs: list,
    k: int = 5
):
    """
    评估向量检索质量
    
    Args:
        questions: 问题列表
        retrieval_function: 检索函数，输入question，返回文档列表
        ground_truth_docs: 每个问题对应的真实相关文档列表
        k: 评估前k个检索结果
    
    Returns:
        dict: 包含Context Precision和Context Recall的评估结果
    """
    # 收集所有检索结果
    all_questions = []
    all_contexts = []
    all_ground_truths = []
    all_answers = []  # 对于检索评估，答案可以为空或占位符
    
    for question, true_docs in zip(questions, ground_truth_docs):
        # 执行检索
        retrieved_docs = retrieval_function(question)[:k]
        
        all_questions.append(question)
        all_contexts.append(retrieved_docs)
        all_ground_truths.append(true_docs)
        all_answers.append("")  # 检索评估不需要实际答案
    
    # 创建Dataset对象
    eval_data = {
        "question": all_questions,
        "answer": all_answers,
        "contexts": all_contexts,
        "ground_truths": all_ground_truths
    }
    dataset = Dataset.from_dict(eval_data)
    
    # 只评估检索相关的指标
    retrieval_metrics = [context_precision, context_recall]
    
    # 执行评估
    evaluation_result = evaluate(
        dataset=dataset,
        metrics=retrieval_metrics
    )
    
    # 提取结果
    precision_scores = evaluation_result["context_precision"]
    recall_scores = evaluation_result["context_recall"]
    
    return {
        "context_precision": {
            "scores": precision_scores.tolist() if hasattr(precision_scores, 'tolist') else list(precision_scores),
            "mean": np.mean(precision_scores),
            "std": np.std(precision_scores)
        },
        "context_recall": {
            "scores": recall_scores.tolist() if hasattr(recall_scores, 'tolist') else list(recall_scores),
            "mean": np.mean(recall_scores),
            "std": np.std(recall_scores)
        }
    }

# 示例：创建检索函数
def simple_retrieval(question: str):
    """简单的检索函数示例"""
    # 实际应该从向量库检索
    # 这里返回示例文档
    all_docs = [
        "RAG（Retrieval-Augmented Generation）是一种结合检索和生成的技术。",
        "RAG的核心思想是在生成答案前，先从知识库中检索相关文档。",
        "RAG可以解决大模型的知识盲区问题，让模型能够访问非公开知识。",
        "向量检索是RAG系统中常用的检索方法。",
        "检索到的文档作为上下文输入给生成模型。"
    ]
    # 简单模拟：根据关键词匹配返回文档
    if "RAG" in question:
        return all_docs[:3]
    else:
        return all_docs[:2]

# 执行检索质量评估
questions = ["RAG是什么？", "什么是向量检索？"]
ground_truths = [
    ["RAG（Retrieval-Augmented Generation）是一种结合检索和生成的技术。",
     "RAG的核心思想是在生成答案前，先从知识库中检索相关文档。"],
    ["向量检索是RAG系统中常用的检索方法。"]
]

retrieval_results = evaluate_retrieval_quality(
    questions=questions,
    retrieval_function=simple_retrieval,
    ground_truth_docs=ground_truths,
    k=5
)

print("=" * 60)
print("检索质量评估结果：")
print("=" * 60)
print(f"\nContext Precision:")
print(f"  平均分数: {retrieval_results['context_precision']['mean']:.4f}")
print(f"  标准差: {retrieval_results['context_precision']['std']:.4f}")
print(f"  各用例分数: {retrieval_results['context_precision']['scores']}")

print(f"\nContext Recall:")
print(f"  平均分数: {retrieval_results['context_recall']['mean']:.4f}")
print(f"  标准差: {retrieval_results['context_recall']['std']:.4f}")
print(f"  各用例分数: {retrieval_results['context_recall']['scores']}")


## 8. 完整示例代码 {#完整示例}

以下是一个完整的RAG系统评估示例，展示了如何评估一个基于FAISS向量库的RAG系统。


In [ ]:
# 完整的RAG系统评估示例
# 注意：这是一个模板，需要根据你的实际RAG系统进行调整

class CompleteRAGEvaluator:
    """完整的RAG评估器（使用新版本RAGAS API）"""
    
    def __init__(self, rag_chain, vector_store, llm=None):
        """
        初始化评估器
        
        Args:
            rag_chain: LangChain RAG链
            vector_store: 向量存储（FAISS等）
            llm: LLM模型（可选，用于生成答案）
        """
        self.rag_chain = rag_chain
        self.vector_store = vector_store
        self.llm = llm
        
        # 定义评估指标（新版本不再使用评估链）
        self.metrics = [
            faithfulness,
            answer_relevancy,
            context_precision,
            context_recall
        ]
    
    def query(self, question: str, k: int = 5):
        """
        执行RAG查询
        
        Args:
            question: 用户问题
            k: 检索文档数量
        
        Returns:
            dict: 包含question, answer, contexts的字典
        """
        # 检索相关文档
        if hasattr(self.vector_store, 'as_retriever'):
            retriever = self.vector_store.as_retriever(search_kwargs={"k": k})
            contexts = retriever.get_relevant_documents(question)
        else:
            contexts = self.vector_store.similarity_search(question, k=k)
        
        # 提取文档内容
        contexts_text = [doc.page_content if hasattr(doc, 'page_content') else str(doc) 
                        for doc in contexts]
        
        # 生成答案
        if self.rag_chain:
            answer = self.rag_chain.invoke(question)
            if hasattr(answer, 'content'):
                answer = answer.content
        else:
            answer = "示例答案"  # 如果没有提供链，返回示例答案
        
        return {
            "question": question,
            "answer": answer,
            "contexts": contexts_text
        }
    
    def evaluate_single(self, question: str, ground_truths: list = None, k: int = 5):
        """
        评估单个查询
        
        Args:
            question: 用户问题
            ground_truths: 标准答案列表（用于Context Recall）
            k: 检索文档数量
        
        Returns:
            dict: 评估结果
        """
        # 执行查询
        result = self.query(question, k=k)
        
        # 准备评估数据
        eval_data = {
            "question": [result["question"]],
            "answer": [result["answer"]],
            "contexts": [result["contexts"]],
            "ground_truths": [ground_truths if ground_truths else [result["answer"]]]
        }
        dataset = Dataset.from_dict(eval_data)
        
        # 执行评估
        evaluation_result = evaluate(
            dataset=dataset,
            metrics=self.metrics
        )
        
        # 提取评估结果
        eval_result = {
            "question": result["question"],
            "answer": result["answer"],
            "contexts": result["contexts"]
        }
        
        for metric in self.metrics:
            metric_name = get_metric_name(metric)  # 使用辅助函数获取指标名称
            try:
                score = evaluation_result[metric_name]
                eval_result[metric_name] = score[0] if isinstance(score, (list, np.ndarray)) else score
            except Exception as e:
                print(f"评估 {metric_name} 时出错: {str(e)}")
                eval_result[metric_name] = None
        
        return eval_result
    
    def evaluate_batch(self, test_dataset: list, k: int = 5):
        """
        批量评估
        
        Args:
            test_dataset: 测试数据集，每个元素包含question和ground_truths
            k: 检索文档数量
        
        Returns:
            DataFrame: 评估结果DataFrame
        """
        # 收集所有查询结果
        all_questions = []
        all_answers = []
        all_contexts = []
        all_ground_truths = []
        
        for test_case in test_dataset:
            result = self.query(test_case["question"], k=k)
            all_questions.append(result["question"])
            all_answers.append(result["answer"])
            all_contexts.append(result["contexts"])
            all_ground_truths.append(test_case.get("ground_truths", [result["answer"]]))
        
        # 创建Dataset
        eval_data = {
            "question": all_questions,
            "answer": all_answers,
            "contexts": all_contexts,
            "ground_truths": all_ground_truths
        }
        dataset = Dataset.from_dict(eval_data)
        
        # 执行批量评估
        evaluation_result = evaluate(
            dataset=dataset,
            metrics=self.metrics
        )
        
        # 转换为DataFrame
        results = []
        for idx in range(len(test_dataset)):
            case_result = {
                "question": all_questions[idx],
                "answer": all_answers[idx]
            }
            
            for metric in self.metrics:
                metric_name = metric_names[metric]
                try:
                    scores = evaluation_result[metric_name]
                    case_result[metric_name] = scores[idx] if idx < len(scores) else None
                except Exception as e:
                    print(f"提取 {metric_name} 分数时出错: {str(e)}")
                    case_result[metric_name] = None
            
            results.append(case_result)
        
        return pd.DataFrame(results)

# 使用示例
print("完整RAG评估器已创建")
print("\n使用方法：")
print("1. 初始化评估器：evaluator = CompleteRAGEvaluator(rag_chain, vector_store)")
print("2. 单个评估：result = evaluator.evaluate_single(question, ground_truths)")
print("3. 批量评估：results_df = evaluator.evaluate_batch(test_dataset)")


In [ ]:
def analyze_evaluation_results(evaluation_df):
    """
    分析评估结果并给出优化建议
    
    Args:
        evaluation_df: 包含评估结果的DataFrame
    
    Returns:
        dict: 分析结果和建议
    """
    analysis = {
        "summary": {},
        "recommendations": []
    }
    
    thresholds = {
        "faithfulness": 0.7,
        "answer_relevancy": 0.7,
        "context_precision": 0.7,
        "context_recall": 0.7
    }
    
    for metric, threshold in thresholds.items():
        if metric in evaluation_df.columns:
            mean_score = evaluation_df[metric].mean()
            analysis["summary"][metric] = {
                "mean": mean_score,
                "threshold": threshold,
                "status": "良好" if mean_score >= threshold else "需要改进"
            }
            
            if mean_score < threshold:
                if metric == "faithfulness":
                    analysis["recommendations"].append(
                        "Faithfulness得分较低：改进prompt，明确要求基于文档回答，避免幻觉"
                    )
                elif metric == "answer_relevancy":
                    analysis["recommendations"].append(
                        "Answer Relevancy得分较低：改进问题理解，优化prompt中的回答要求"
                    )
                elif metric == "context_precision":
                    analysis["recommendations"].append(
                        "Context Precision得分较低：改进embedding模型，增加rerank步骤"
                    )
                elif metric == "context_recall":
                    analysis["recommendations"].append(
                        "Context Recall得分较低：增加检索数量k，使用混合检索策略"
                    )
    
    return analysis

# 如果有评估结果，进行分析
if 'evaluation_df' in locals() and not evaluation_df.empty:
    analysis = analyze_evaluation_results(evaluation_df)
    
    print("=" * 60)
    print("评估结果分析：")
    print("=" * 60)
    
    print("\n指标概览：")
    for metric, info in analysis["summary"].items():
        status_icon = "✅" if info["status"] == "良好" else "⚠️"
        print(f"{status_icon} {metric:20s}: {info['mean']:.4f} (阈值: {info['threshold']:.2f}) - {info['status']}")
    
    if analysis["recommendations"]:
        print("\n优化建议：")
        for i, rec in enumerate(analysis["recommendations"], 1):
            print(f"{i}. {rec}")
    else:
        print("\n✅ 所有指标都达到阈值，系统表现良好！")
else:
    print("请先运行批量评估以生成分析结果")


### 9.3 最佳实践总结

1. **评估数据集构建**：
   - 包含不同类型的问题（简单事实、复杂推理、多跳问题）
   - 确保有足够的ground truth文档用于Context Recall评估
   - 定期更新评估数据集

2. **评估频率**：
   - 每次模型或prompt更新后都应重新评估
   - 建立定期评估机制（如每周或每月）
   - 记录评估历史，追踪改进效果

3. **指标选择**：
   - 根据应用场景选择重点关注的指标
   - 检索系统重点关注Context Precision和Recall
   - 生成系统重点关注Faithfulness和Answer Relevancy

4. **优化迭代**：
   - 根据评估结果定位问题
   - 逐步优化（先优化检索，再优化生成）
   - 使用A/B测试验证优化效果

### 9.4 常见问题

**Q1: RAGAS评估需要多长时间？**
- A: 每个评估需要调用LLM，速度取决于LLM的响应时间。批量评估可以并行处理以提高效率。

**Q2: 评估结果不稳定怎么办？**
- A: 使用temperature=0确保一致性；多次评估取平均值；检查评估数据质量。

**Q3: 如何提高评估准确性？**
- A: 使用更强大的LLM作为评估器（如GPT-4）；确保ground truth质量；增加评估数据量。

**Q4: 可以自定义评估指标吗？**
- A: RAGAS提供了基础的评估框架，可以通过LangChain自定义评估链来实现特定的评估逻辑。

---

## 总结

本教程介绍了如何使用RAGAS框架评估RAG系统的向量检索性能。通过Faithfulness、Answer Relevancy、Context Precision和Context Recall四个核心指标，我们可以全面了解RAG系统在检索和生成方面的表现。

**关键要点**：
- ✅ RAGAS提供了无参考评估能力，降低评估成本
- ✅ 四个核心指标覆盖检索和生成的关键维度
- ✅ 可以与LangChain无缝集成
- ✅ 支持单个和批量评估

**下一步**：
- 将RAGAS评估集成到你的RAG系统中
- 建立评估数据集和定期评估机制
- 根据评估结果持续优化系统性能
